# Instructions for setting up an AWS S3 bucket for your project

This set of instructions will walk through how to setup an AWS S3 bucket for a specific project and how to configure that bucket to allow all members of the project team to have access.

## Create an AWS account and S3 bucket

The first step is to create an AWS account that will be billed to your particular project. This can be done using these [instructions](https://aws.amazon.com/premiumsupport/knowledge-center/create-and-activate-aws-account/).


## Create AWS S3 bucket
Within your new AWS account, create an new S3 bucket:

1. Open the AWS S3 console (https://console.aws.amazon.com/s3/)
1. From the navigation pane, choose **Buckets**
1. Choose **Create bucket**
1. Name the bucket and select us-west-2 for the region
1. Leave all other default options
1. Click **Create Bucket**

## Create a user
Within the same AWS account, create a new IAM user:

1. On the **AWS Console Home** page, select the IAM service
1. In the navigation pane, select **Users** and then select **Add users**
1. Name the user and click Next
1. Attach policies directly
1. Do not select any policies
1. Click Next
1. Create user

Once the user has been created, find the user's ARN and copy it.

Now, create access keys for this user:

1. Select **Users** and click the user that you created
1. Open the Security Credentials tab
1. Create access key
1. Select **Command Line Interface (CLI)**
1. Check the box to agree to the recommendation and click **Next**
1. Leave the tag blank and click **Create access key**
1. IMPORTANT: Copy the access key and the secret access key. This will be used later.

## Create the bucket policy
Configure a policy for this S3 bucket that will allow the newly created user to access it.

1. Open the AWS S3 console (https://console.aws.amazon.com/s3/)
1. From the navigation pane, choose **Buckets**
1. Select the new S3 bucket that you created
1. Open the Permissions tab
1. Add the following bucket policy, replacing `USER_ARN` with the ARN that you copied above and `BUCKET_ARN` with the bucket ARN, found on the Edit bucket policy page on the AWS console:

```
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "ListBucket",
            "Effect": "Allow",
            "Principal": {
                "AWS": "USER_ARN"
            },
            "Action": "s3:ListBucket",
            "Resource": "BUCKET_ARN"
        },
        {
            "Sid": "AllObjectActions",
            "Effect": "Allow",
            "Principal": {
                "AWS": "USER_ARN"
            },
            "Action": "s3:*Object",
            "Resource": "BUCKET_ARN/*"
        }
    ]
}
```

# Configure local AWS profile

To access the bucket from a "local" computer (for example, a local computer or a JupyterHub that is running on AWS), an AWS profile must be configured with the access keys to the new S3 bucket.

This can be done manually but the easiest way to do it is by using [AWS Command Line Interface (CLI)](https://aws.amazon.com/cli/).

On the local computer:
1. Open a terminal
1. Install AWS CLI (if using the `conda` package manager, you can install like this: `conda install -c conda-forge awscli`)
1. Configure the AWS profile with the keys from above and give your profile a new profile name:
- `aws configure --profile PROFILE_NAME`
- Enter the access key and secret access key
- Enter `us-west-2` for the region and `json` for format

In the code examples below, the profile name is `icesat2`. Replace this with the profile name that you used.

***

# Example code

Below is code that can be used to read from and write to the S3 bucket in order to test that the bucket and local AWS profile have been configured correctly. In the examples below, we are accessing a bucket called `gris-outlet-glacier-seasonality-icesat2`. Replace this name with the name of the S3 bucket that you have created.

## Reading from the S3 bucket

### Example: ls bucket using s3fs

In [ ]:
import s3fs
s3 = s3fs.S3FileSystem(anon=False, profile='icesat2')
s3.ls('gris-outlet-glacier-seasonality-icesat2')


### Example: open HDF5 file using xarray

In [ ]:
import s3fs
import xarray as xr

fs_s3 = s3fs.core.S3FileSystem(profile='icesat2')

s3_url = 's3://gris-outlet-glacier-seasonality-icesat2/ssh_grids_v2205_1992101012.nc'
s3_file_obj = fs_s3.open(s3_url, mode='rb')
ssh_ds = xr.open_dataset(s3_file_obj, engine='h5netcdf')
print(ssh_ds)


In [ ]:
import s3fs

import xarray as xr

import hvplot.xarray
import holoviews as hv

fs_s3 = s3fs.core.S3FileSystem(profile='icesat2')

s3_url = 's3://gris-outlet-glacier-seasonality-icesat2/ssh_grids_v2205_1992101012.nc'
s3_file_obj = fs_s3.open(s3_url, mode='rb')
ssh_ds = xr.open_dataset(s3_file_obj, engine='h5netcdf')
#ssh_ds

ssh_da = ssh_ds.SLA
#ssh_da

ssh_da.hvplot.image(x='Longitude', y='Latitude', cmap='Spectral_r', geo=True, tiles='ESRI', global_extent=True)


### Example: read a geotiff using rasterio

In [ ]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt

session = rasterio.env.Env(profile_name='icesat2')

url = 's3://gris-outlet-glacier-seasonality-icesat2/out.tif'

with session:
    with rasterio.open(url) as ds:
        print(ds.profile)
        band1 = ds.read(1)
        
band1[band1==-9999] = np.nan
plt.imshow(band1)
plt.colorbar()


## Writing to the S3 bucket

In [ ]:
s3 = s3fs.core.S3FileSystem(profile='icesat2')

with s3.open('gris-outlet-glacier-seasonality-icesat2/new-file', 'wb') as f:
    f.write(2*2**20 * b'a')
    f.write(2*2**20 * b'a') # data is flushed and file closed

s3.du('gris-outlet-glacier-seasonality-icesat2/new-file')